In [1]:
import socket
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
HOST = "127.0.0.1"

In [3]:
class NoiseServer:
    def __init__(self, noise_server_port, final_server_port, block_size=1024):
        self.final_server_port = final_server_port
        self.noise_server_port = noise_server_port
        self.block_size = block_size
        
        self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket_server.bind((HOST, noise_server_port))
        self.socket_server.listen()
         # Sends image
        self.socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    
    def run(self):
        # Получение оригинального изображения
        img_orig = self.recv_image()
        # Добавление шума на оригинальное изображение
        img_noise = self.add_noise(img_orig)
        with self.socket_client as s:
            print(f'Waiting for connetion with port {self.final_server_port}.')
            s.connect((HOST, self.final_server_port))
            print(f'Connected. Port:{self.final_server_port}. Sending images.')
            # Отправка изображения с шумом
            self.send_image(img_noise, s)
            # Отправка оригинального изображения
            self.send_image(img_orig, s)
 

    def recv_image(self):
        with self.socket_server as s:
            print(f'Waiting for connection on port {self.noise_server_port}.')
            conn, addr = s.accept()
            print(f'Connected. Port:{self.noise_server_port}.')

            # Получение размера изображения
            h_bytes = conn.recv(4)
            h = int.from_bytes(h_bytes, byteorder='big')
            w_bytes = conn.recv(4)
            w = int.from_bytes(w_bytes, byteorder='big')
            print(f'Размер изображения h = {h} w = {w}.')

            # Получение изображения
            n = h * w * 3
            n_iters = n // self.block_size
            n_remainder = n - n_iters * self.block_size
            img_bytes = []
            for i in range(n_iters):
                block = conn.recv(self.block_size)
                img_bytes.append(block)
            if n_remainder > 0:
                block = conn.recv(n_remainder)
                img_bytes.append(block)
            print("Image was received.")

            img_bytes_b = b''.join(img_bytes)
            img_np = np.frombuffer(img_bytes_b, dtype='uint8')
            img = img_np.reshape(h, w, 3)
        return img
    
    def send_image(self, img, s):  
        # Отправка размера изображения
        h, w, _ = img.shape
        n = h * w * 3  # size of bytes array
        h_bytes = h.to_bytes(length=4, byteorder='big')
        w_bytes = w.to_bytes(length=4, byteorder='big')
        img_bytes = img.tobytes()
        assert s.sendall(h_bytes) is None, print("Ошибка отправки h_bytes: %s", e)
        assert s.sendall(w_bytes) is None, print("Ошибка отправки w_bytes: %s", e)

        # Отправка изображения
        n_iters = n // self.block_size
        n_remainder = n - n_iters * self.block_size
        for i in range(n_iters):
            block = img_bytes[self.block_size*i:(i+1)*self.block_size]
            assert s.sendall(block) is None, print("Ошибка отправки изображения: %s", e)

        if n_remainder > 0:
            block = img_bytes[-n_remainder:]
            assert s.sendall(block) is None, print("Ошибка отправки изображения: %s", e)
        print("Изображение успешно отправлено.")
        
    def add_noise(self, image):
        img = image.copy()
        h, w, c = img.shape
        # Генерация шума
        gaussian = np.random.normal(0, 1, img.size)
        gaussian = gaussian.reshape(h, w, c).astype('uint8')
        # Добавление шума на изображение
        img_gaussian = cv2.add(img, gaussian)
        return img_gaussian

In [4]:
noiseserver = NoiseServer(noise_server_port=5065, final_server_port=5066)

In [5]:
img = noiseserver.run()

Waiting for connection on port 5065.
Connected. Port:5065.
Размер изображения h = 1280 w = 2048.
Image was received.
Waiting for connetion with port 5066.
Connected. Port:5066. Sending images.
Изображение успешно отправлено.
Изображение успешно отправлено.
